In [ ]:
import pandas as pd

# Forst Cover Data

http://archive.ics.uci.edu/ml/datasets/Covertype

Data Set Information:

Predicting forest cover type from cartographic variables only (no remotely sensed data). The actual forest cover type for a given observation (30 x 30 meter cell) was determined from US Forest Service (USFS) Region 2 Resource Information System (RIS) data. Independent variables were derived from data originally obtained from US Geological Survey (USGS) and USFS data. Data is in raw form (not scaled) and contains binary (0 or 1) columns of data for qualitative independent variables (wilderness areas and soil types).

This study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. These areas represent forests with minimal human-caused disturbances, so that existing forest cover types are more a result of ecological processes rather than forest management practices.

Some background information for these four wilderness areas: Neota (area 2) probably has the highest mean elevational value of the 4 wilderness areas. Rawah (area 1) and Comanche Peak (area 3) would have a lower mean elevational value, while Cache la Poudre (area 4) would have the lowest mean elevational value.

As for primary major tree species in these areas, Neota would have spruce/fir (type 1), while Rawah and Comanche Peak would probably have lodgepole pine (type 2) as their primary species, followed by spruce/fir and aspen (type 5). Cache la Poudre would tend to have Ponderosa pine (type 3), Douglas-fir (type 6), and cottonwood/willow (type 4).

The Rawah and Comanche Peak areas would tend to be more typical of the overall dataset than either the Neota or Cache la Poudre, due to their assortment of tree species and range of predictive variable values (elevation, etc.) Cache la Poudre would probably be more unique than the others, due to its relatively low elevation range and species composition.

Attribute Information:

Given is the attribute name, attribute type, the measurement unit and a brief description. The forest cover type is the classification problem. The order of this listing corresponds to the order of numerals along the rows of the database.

Name / Data Type / Measurement / Description

Elevation / quantitative /meters / Elevation in meters
Aspect / quantitative / azimuth / Aspect in degrees azimuth
Slope / quantitative / degrees / Slope in degrees
Horizontal_Distance_To_Hydrology / quantitative / meters / Horz Dist to nearest surface water features
Vertical_Distance_To_Hydrology / quantitative / meters / Vert Dist to nearest surface water features
Horizontal_Distance_To_Roadways / quantitative / meters / Horz Dist to nearest roadway
Hillshade_9am / quantitative / 0 to 255 index / Hillshade index at 9am, summer solstice
Hillshade_Noon / quantitative / 0 to 255 index / Hillshade index at noon, summer soltice
Hillshade_3pm / quantitative / 0 to 255 index / Hillshade index at 3pm, summer solstice
Horizontal_Distance_To_Fire_Points / quantitative / meters / Horz Dist to nearest wildfire ignition points
Wilderness_Area (4 binary columns) / qualitative / 0 (absence) or 1 (presence) / Wilderness area designation
Soil_Type (40 binary columns) / qualitative / 0 (absence) or 1 (presence) / Soil Type designation
Cover_Type (7 types) / integer / 1 to 7 / Forest Cover Type designation


In [ ]:
column_names = (
["Elevation",
"Aspect",
"Slope",
"Horizontal_Distance_To_Hydrology",
"Vertical_Distance_To_Hydrology",
"Horizontal_Distance_To_Roadways",
"Hillshade_9am",
"Hillshade_Noon",
"Hillshade_3pm",
"Horizontal_Distance_To_Fire_Points"]
    + ['WE{}'.format(i) for i in range(4)]
    + ['ST{}'.format(i) for i in range(40)]
    + ['Cover'])

In [ ]:
cover_data = pd.read_csv('data/covtype.data.gz', names=column_names)

In [ ]:
cover_data.head()

In [ ]:
%matplotlib inline

In [ ]:
cover_data.Elevation.plot.hist(bins=30)

In [ ]:
cover_data.WE2.hist()

In [ ]:
cover_data.groupby('Cover').Cover.count() / cover_data.shape[0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sample = cover_data.sample(10000)
for k, group in sample.groupby('Cover'):
    plt.scatter(group.Elevation, group.Slope)

In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(cover_data, test_size=0.3)

In [ ]:
means, pis, labels, Sigmas = [], [], [], []
N, p = train.shape
p -= 1
Sigma = np.zeros([p, p])
for label, data in train.groupby('Cover'):
    labels.append(label)
    Nk, _ = data.shape
    print Nk, N, label
    pis.append(Nk / float(N))
    mu = data.mean()
    means.append(mu[:-1])
    xn = (data - mu).values[:,:-1]
    S = np.zeros([p, p])
    for i in range(Nk):
        S += np.dot(xn[i:i+1,:].T, xn[i:i+1,:])
    Sigmas.append(S / (Nk - 1))
    Sigma += S
Sigma /= float(N - len(labels))

In [ ]:
Sigmainv = np.linalg.inv(Sigma)

In [ ]:
pis

In [ ]:
def delta(x, mu, pi):
    return (np.dot(np.dot(x, Sigmainv), mu)
            - 0.5 * np.dot(np.dot(mu.T, Sigmainv), mu)
            + np.log(pi))

In [ ]:
def lda(x):
    return np.argmax(np.array([delta(x, mu, pi) for mu, pi in zip(means, pis)]).T, axis=1)

In [ ]:
labels

In [ ]:
(lda(test[column_names[:-1]]) + 1 == test.Cover).mean()

In [ ]:
for l, X in test.groupby('Cover'):
    print l, (lda(X[column_names[:-1]]) + 1 == l).mean(), float(len(X)) / len(test)

In [ ]:
Si = np.linalg.inv(0.5*Sigmas[0] + 0.5*Sigma)

In [ ]:
def delta_r(x, mu, pi, logS, Sinv):
    return (-0.5 * logS
            - 0.5 * np.sum(np.dot((x-mu), Sinv) * (x-mu), axis=1)
            + np.log(pi))

In [ ]:
def rda(x, alpha):
    Sigmas_r = [alpha*S + (1-alpha)*Sigma for S in Sigmas]
    eigs = [np.linalg.eig(S)[0] for S in Sigmas_r]
    logs = [np.sum(np.log(e[e>1e-12])) for e in eigs]
    Sinv_r = [np.linalg.inv(S) for S in Sigmas_r]
    vals = np.array([delta_r(x, mu, pi, logS, Sinv) for mu, pi, Sinv, logS 
                               in zip(means, pis, Sinv_r, logs)]).T
    return np.argmax(vals, axis=1)

In [ ]:
(rda(test[column_names[:-1]], 0) + 1 == test.Cover).mean()

In [ ]:
(rda(test[column_names[:-1]], 0.8) + 1 == test.Cover).mean()

In [ ]:
for l, X in test.groupby('Cover'):
    print l, (rda(X[column_names[:-1]], 0.2) + 1 == l).mean(), float(len(X)) / len(test)

In [ ]:
for l, X in test.groupby('Cover'):
    print l, (rda(X[column_names[:-1]], 0.8) + 1 == l).mean(), float(len(X)) / len(test)

In [ ]:
rs = np.arange(0.1, 1, 0.1)
hits = [(rda(test[column_names[:-1]], r) + 1 == test.Cover).mean() for r in rs]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(rs, hits)
plt.xlabel(r'$\alpha$')
plt.ylabel('hit rate')

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
sk_model = LinearDiscriminantAnalysis().fit(train[column_names[:-1]], train.Cover)

In [ ]:
sk_hits = sk_model.predict(test[column_names[:-1]]) == test.Cover

In [ ]:
sk_hits.mean()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
import time

In [ ]:
deltas = []
Ns = np.arange(1e4, 6e4, 1e4)
for N in Ns:
    X = train.sample(int(N))
    start = time.time()
    lr = LogisticRegression().fit(X[column_names[:-1]], X.Cover)
    deltas.append(time.time() - start)

In [ ]:
deltasLDA = []
for N in Ns:
    X = train.sample(int(N))
    start = time.time()
    lda = LinearDiscriminantAnalysis().fit(X[column_names[:-1]], X.Cover)
    deltasLDA.append(time.time() - start)

In [ ]:
plt.plot(Ns, deltas)
plt.plot(Ns, deltasLDA)